In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/New_datasets/gnq

/content/drive/.shortcut-targets-by-id/1Z4iGm8N8wfD0bEVxyzfrnT5o0V5X8xnV/New_datasets/gnq


#Setting up libraries

In [ ]:
# pip installing all the libraries
%%capture
!pip install jax
!pip install flax
!pip install sentencepiece
!pip install transformers
!pip install -U sentence-transformers



In [ ]:
#All the imports
from google.colab import output
import pandas as pd
import pickle
import jax
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import jax.numpy as jnp
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
import torch
import urllib3
import requests
import re
import pickle
from googlesearch import search
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import timeit
import nltk
import textwrap
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
from transformers import FlaxBigBirdForQuestionAnswering, BigBirdTokenizer
# import gradio as gr
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import urlparse




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Utilites and Functions


In [ ]:

torch.set_grad_enabled(False)
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

In [ ]:
model_dpr = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')


In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

In [ ]:
import inspect
from pprint import pprint

pprint(inspect.getfullargspec(q_tokenizer))

FullArgSpec(args=['self', 'text', 'text_pair', 'add_special_tokens', 'padding', 'truncation', 'max_length', 'stride', 'is_split_into_words', 'pad_to_multiple_of', 'return_tensors', 'return_token_type_ids', 'return_attention_mask', 'return_overflowing_tokens', 'return_special_tokens_mask', 'return_offsets_mapping', 'return_length', 'verbose'], varargs=None, varkw='kwargs', defaults=(None, True, False, False, None, 0, False, None, None, None, None, False, False, False, False, True), kwonlyargs=[], kwonlydefaults=None, annotations={'return': <class 'transformers.tokenization_utils_base.BatchEncoding'>, 'text': typing.Union[str, typing.List[str], typing.List[typing.List[str]]], 'text_pair': typing.Union[str, typing.List[str], typing.List[typing.List[str]], NoneType], 'add_special_tokens': <class 'bool'>, 'padding': typing.Union[bool, str, transformers.file_utils.PaddingStrategy], 'truncation': typing.Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy], 'max_length': t

In [ ]:
inspect.getfullargspec(ctx_encoder
                      )

FullArgSpec(args=['self'], varargs='input', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [ ]:
j = ctx_encoder(j).pooler_output
k = q_encoder(k).pooler_output

In [ ]:
l = k@(j.T)

In [ ]:
def xyz(ques, cont):

  ctx_tokens = ctx_tokenizer(cont, max_length = 512, truncation = True, return_tensors='pt')["input_ids"]
  q_tokens = q_tokenizer(ques,return_tensors='pt')["input_ids"]
  ctx_encoded = ctx_encoder(ctx_tokens).pooler_output
  q_encoded = q_encoder(q_tokens).pooler_output
  similarity = (q_encoded@(ctx_encoded.T)).item()

  return similarity

In [ ]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
len(cl)

28

In [ ]:

def combine_strs(list_of_str):
  ans_list = []
  i=0
  deli.deli_2()
  print('Entering combine_strs')
  while i < (len(list_of_str)):
    j = i
    temp_str =""
    while(j<len(list_of_str) and len(temp_str.split(" "))<256):
      temp_str+=list_of_str[j]+" "
      # print(temp_str)
      j+=1
    ans_list.append(temp_str)
    i=j
  print('exiting combine_strs -----' , len(ans_list))
  deli.deli_2()
  return ans_list

def rank_pass(q,ct,alpha=0.5):
  cl = nltk.sent_tokenize(ct)


  print(len(cl))
  if len(cl) > 3000:
    cl = combine_strs(cl)

  similarity_scores = [xyz(q,a) for i,a in enumerate(cl)]
  # similarity_scores = sorted(similarity_scores, reverse = True)

  similarity_scores = np.array(similarity_scores)
  # print(similarity_scores)
  mean = np.mean(similarity_scores)
  std = np.std(similarity_scores) * alpha
  thresh1 = mean+std
  # s1 = np.where(similarity_scores >= thresh2,1,0)
  s2 = np.where(similarity_scores >= thresh1,1,0)
  s = np.where(s2 == 1)[0]
  # print(s)
  cl_list = [cl[j] for j in s]

  return " ".join(cl_list)

def rank_passages_3(df_c,top_k=9):
  df_c=df_c.fillna("")
  df_ans = pd.DataFrame()
  # len_array=[]
  for i, k in df_c.iterrows():
    row_dict={}
    for j in range(10):
      print("Index",i,j)
      passage_ranker=[]
      q=df_c['query'].iloc[i]

      content= df_c[str(j) +"_Content"].iloc[i]
      # printw("Question "+str(i) +":" + q)
      # printw("Content "+str(j) +":" + df_c[str(j) +"_Link"].iloc[i])
      if(content==" this is a meaningless content due to repetition and timeout as well"):
        row_dict[str(j)] =" this is a meaningless content due to repetition and timeout as well"
        continue
      start = timeit.timeit()
      if i == 58 and j ==6:
        row_dict[str(j)] = " this is a meaningless content due to repetition and timeout as well"
        continue
      row_dict[str(j)] = rank_pass(q,content)
      end = timeit.timeit()
      print(end-start)

    df_ans  = df_ans.append(row_dict,ignore_index=True)
  # print(make_histogram(len_array))
  return df_ans

In [ ]:
#Word Wrap
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
#utils
class deli:
  @staticmethod
  def deli_1():
    print("_______________ this is a delimiter _______________")
  @staticmethod
  def deli_2():
    print("_______________________________________________________________________________________________")
  @staticmethod
  def deli_3():
    print("========================================================================")
def list_to_str(some_list):
  return " ".join(some_list)
def split_into_sent(content):
  a_list = nltk.tokenize.sent_tokenize(content)
  return a_list

def harmonic_mean(a):
  b=a[1]
  a=a[0]
  if a==0 or b==0:
    return 0
  else:
    return 2*a*b/(a+b)
import re
CLEANR = re.compile('<.*?>')
spaces = re.compile(' +')
# delimiters = re.compile('')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, ' ', raw_html)
  cleantext = re.sub(spaces, ' ', cleantext)
  return cleantext

def simplify(st):
  temp = cleanhtml(st)
  temp = temp.replace('\n',' ')
  temp = temp.replace('\t',' ')
  return temp


def printw(f):
  wrapper = textwrap.TextWrapper(width=180)

  word_list = wrapper.wrap(text=f)

# Print each line.
  for element in word_list:
    print(element)

def make_histogram(y):
  hist= {}
  for i in y:
    hist[i] = hist.get(i,0)+1
  return hist
def combine_strs(list_of_str):
  ans_list = []
  i=0
  while i < (len(list_of_str)):
    j = i
    temp_str =""
    while(j<len(list_of_str) and len(temp_str.split(" "))<256):
      temp_str+=list_of_str[j]+" "
      # print(temp_str)
      j+=1
    ans_list.append(temp_str)
    i=j
  return ans_list



#Define Models


In [ ]:
%%capture
FLAX_MODEL_ID = "vasudevgupta/flax-bigbird-natural-questions"
model_qa = FlaxBigBirdForQuestionAnswering.from_pretrained(FLAX_MODEL_ID, block_size=64, num_random_blocks=3)
tokenizer_qa = BigBirdTokenizer.from_pretrained(FLAX_MODEL_ID)

@jax.jit
def forward(*args, **kwargs):
  return model_qa(*args, **kwargs)

def get_answer(question, context):

    encoding = tokenizer(question, context, return_tensors="jax", max_length=4096, padding="max_length", truncation=True)
    start_scores, end_scores = forward(**encoding).to_tuple()

    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

    answer_tokens = all_tokens[jnp.argmax(start_scores): jnp.argmax(end_scores)+1]
    answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

    return answer
def run_code():
  question = input("Type your question here : ")
  context = input("Paste the context here : ")
  ans = get_answer(question, context)
  print("Lets see :- ")
  print(ans)


Some weights of the model checkpoint at vasudevgupta/flax-bigbird-natural-questions were not used when initializing FlaxBigBirdForQuestionAnswering: {('bert', 'pooler', 'bias'), ('cls', 'bias'), ('bert', 'pooler', 'kernel'), ('cls', 'kernel')}
- This IS expected if you are initializing FlaxBigBirdForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBigBirdForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_answer_1(question, context):
  # print("I am in this function")
  if(context==" this is a meaningless content due to repetition and timeout as well"):
    return ("No Answer due to meaningless content due to repetition and timeout as well",(0,0))

  encoding = tokenizer_qa(question, context, return_tensors="jax", max_length=4096, padding="max_length", truncation=False)
  start_scores, end_scores = forward(**encoding).to_tuple()
  # print(start_scores.shape, end_scores)
  # Let's take the most likely token using `argmax` and retrieve the answer
  all_tokens = tokenizer_qa.convert_ids_to_tokens(encoding["input_ids"][0].tolist())
  # print(all_tokens)
  g = (jnp.max(start_scores), jnp.max(end_scores))
  start_token = jnp.argmax(start_scores)
  end_token =  jnp.argmax(end_scores)
  answer_tokens = all_tokens[start_token: end_token+1]
  answer = tokenizer_qa.decode(tokenizer_qa.convert_tokens_to_ids(answer_tokens))
  answer=answer.replace("[SEP]", "")
  answer=answer.replace("[CLS]", "")
  # print(start_token, end_token)
  if(answer==""):
    # print("Stuck here")
    # print(end_scores.shape)
    end_token_ = jax.numpy.argsort((end_scores[0]))
    # print(end_token_, end_scores, "Sanity Check")
    # pprint(list(end_scores[0]))
    if(len(end_token_)==1):
      answer = "No Meaningful Answer due to lack of content only 1 word"
    else:
      # print("I am here")
      end_token = end_token_[-2]
      g = (g[0],end_scores[0][end_token_[-2]])
      answer_tokens = all_tokens[start_token: end_token+1]
      answer = tokenizer_qa.decode(tokenizer_qa.convert_tokens_to_ids(answer_tokens))
      answer=answer.replace("[SEP]", "")
      answer=answer.replace("[CLS]", "")

    # print(g, ((start_scores), (end_scores)))
  answer=answer.replace("⁇", "")
  if(answer =="" or answer ==" "):
    g = (1 if g[0]==0 else g[0],0)
  # print("I am leaving this function")
  return (answer,g)

#Crawler


In [ ]:
def google_search(q, n, ls):
  for j in search(q, tld="com", num=10, stop=10, pause=2):
      ls.append(j)


def scraper(query_list,n_links=10):
  df = pd.DataFrame()
  count_F=0
  for i,q in enumerate(query_list):
    print("i->", i)
    row =dict()
    q = q[0]
    row['query'] = q
    ls = []
    unique_url = set()
    print(q[0])
    google_search( q, n_links ,ls)
    # print(ls)
    for j,url in enumerate(ls):
      print("\tj->", j)
      # webpage = requests.get(url)
      # webpage= webpage.content.encode('utf-8')
      domain = urlparse(url).netloc
      if domain in unique_url:
        print("same domain explored before")
        pass
      unique_url.add(domain)
      try:
          http_pool = urllib3.connection_from_url(url)
          r = http_pool.urlopen('GET',url)
          content= r.data.decode('utf-8')

      except:
          try:
              webp = requests.get(url)
              content = webp.content.encode('utf-8')
          except:
              content= "This is a meaningless content   "
              print("F")
              count_F+=1
              pass



      soup_obj = BeautifulSoup(content, 'lxml')

      for script in soup_obj(["script", "style"]):
          script.extract()

      p1  = []
      for para in soup_obj.find_all(["p","li"]):
        p1.append(para.get_text()+" ")
      row[str(j)+"_Content"] = "\n".join(p1)
      row[str(j)+"_Link"] = url
    df = df.append(row, ignore_index=True)
  print(count_F)
  return df


In [ ]:
#google search api +
def scraper(query_list,n_links=10):
  df = pd.DataFrame()
  for i,q in enumerate(query_list):
    print("i", i)
    row =dict()
    row['query'] = q[0]
    ls = []

    google_search(q,n_links, ls)
    for j,url in enumerate(ls):
      print("\tj", j)
      webpage = requests.get(url)
      # webpage= webpage.content.encode('utf-8')
      if (i,j) in [(0,8)]:
        try:
          decoded_webpage = webpage.content.decode('utf-8')
        except:
          print(i,j,"checked cases")
          pass
      else:
        decoded_webpage = webpage.content.decode('utf-8')

      soup_obj = BeautifulSoup(decoded_webpage, 'lxml')

      for script in soup_obj(["script", "style"]):
          script.extract()

      p1  = []
      for para in soup_obj.find_all(["p","li"]):
        p1.append(para.get_text()+" ")
      row[str(j)+"_Content"] = "\n".join(p1)
      row[str(j)+"_Link"] = url
    df = df.append(row, ignore_index=True)
  return df





In [ ]:
df_a = pd.read_csv('extracted_content_1_60.csv')
df_b = rank_passages_3(df_a)

Index 0 0
1
-0.003884965999986889
Index 0 1
5
-0.0024499410000089483
Index 0 2
0
-5.808499997783656e-05
Index 0 3
227
-0.006494809000002988
Index 0 4
37
-8.169999944129813e-06
Index 0 5
17
-0.0014336880000200836
Index 0 6
125
-0.0022198949999392426
Index 0 7
134
-0.0005923049999978502
Index 0 8
80
-0.0027979410000398275
Index 0 9
472
-0.001057256999956735
Index 1 0
179
0.0003523110000287488
Index 1 1
388
-0.001554157999976269
Index 1 2
36
-0.0018037659999663447
Index 1 3
12
0.0007494000001315726
Index 1 4
18
-0.0032870039999579603
Index 1 5
65
-0.0009976590000633223
Index 1 6
75
-0.00290102599990405
Index 1 7
78
0.001274169999987862
Index 1 8
0
-0.0006610300000602365
Index 1 9
1
0.002160174000096049
Index 2 0
90
-0.005975260999889542
Index 2 1
217
0.0003475059999118457
Index 2 2
31
-0.0018800410000494594
Index 2 3
33
-0.001999600999965878
Index 2 4
38
-8.940500003973284e-05
Index 2 5
41
0.00020852700004070357
Index 2 6
98
-0.001077677000012045
Index 2 7
26
-0.0026540310000200407
Index 

In [ ]:
df_b.to_csv('dpr_faiss_60.csv', index = False)

#Document Shortening Algorithm


##TFIDF + Cosine similarity + window size =2

In [ ]:
# #usage

# TFIDF + Cosine similarity + window size =2
# def select_sent(query,sent_list,thresh=0.2):
#   vect = TfidfVectorizer(min_df=1, stop_words="english")
#   try:
#     tfidf = vect.fit(sent_list)
#   except:
#     return [(i,k) for i,k in enumerate(sent_list)]

#   ans=[]
#   for i,sent in enumerate(sent_list):
#     s = [query, sent]
#     tfidf_1 = tfidf.transform(s)
#     pairwise_similarity = tfidf_1 * tfidf_1.T
#     simma = ( pairwise_similarity.toarray())[0,1]
#     if(simma>thresh):
#       ans.append((i,sent))
#   return ans

# def list2para(ans_list,sent_list,window_size=2):

#   d= set()
#   len_sent= len(sent_list)
#   for i,k in ((ans_list)):
#     for j in range(-window_size,window_size+1):
#       if(i+j>=0 and i+j<len_sent):
#         d.add((i+j,sent_list[i+j]))
#   d = list(d)
#   d =sorted(d)
#   return list_to_str([j[1] for j in list(d)])


# def answer_2(df):
#   df=df.fillna("")
#   list_1=[]
#   for i, k in df.iterrows():
#     q = k['query']
#     print(i)
#     list_temp=[]
#     for j in range( 10):
#       c = k[str(j)+"_Content"]
#       # c = make_tuple()
#       c = simplify(c)


#       sent_list = split_into_sent(c)

#       ans_list = select_sent(q,sent_list,thresh= 0.15)

#       l2p = list2para(ans_list,sent_list)
#       ans= get_answer_1(q,l2p)
#       list_temp.append((harmonic_mean(ans[1]),ans[0],j))
#     list_temp_1 = sorted(list_temp)
#     list_temp_1.reverse()
#     list_temp_temp = [(ii[1],ii[2]) for ii in list_temp_1]
#     list_temp = [q,k['ans']]
#     list_temp.extend(list_temp_temp)
#     list_temp=list_temp[:7]
#     list_1.append(list_temp)
#   return list_1

##Passage Ranking with MS Marco


In [ ]:
model_n='cross-encoder/ms-marco-TinyBERT-L-2'
model_pr = AutoModelForSequenceClassification.from_pretrained(model_n)
tokenizer_pr = AutoTokenizer.from_pretrained(model_n)

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def score_for_one(q, list_of_passages):

  features=tokenizer_pr([q],list_of_passages,padding =True, truncation = True, return_tensors='pt')

  model_pr.eval()
  with torch.no_grad():
      scores = model_pr(**features).logits
      return scores

In [ ]:
# bullshit implementation

# def rank_passages(q_list,df ):
#   df =df.fillna("")
#   df_1= pd.DataFrame()
#   for i in range(10):
#     features = tokenizer(q_list, list(df[str(i)+"_Content"]),padding =True, truncation = False, return_tensors='pt')
#     model_pr.eval()
#     with torch.no_grad():
#       scores =  model_pr(**features).logits
#       df_1['scores_'+str(i)] = list(scores)
#   return df_1


In [ ]:
#threshinf at -5? should help

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def rank_passages_1(df_c,top_k=9):
  df_c=df_c.fillna("")
  df_ans = pd.DataFrame()
  len_array=[]
  for i, k in df_c.iterrows():
    row_dict={}
    for j in range(10):
      print(i,j)
      passage_ranker=[]
      q=df_c['query'].iloc[i]

      content= df_c[str(j) +"_Content"].iloc[i]
      printw("Question "+str(i) +":" + q)
      printw("Content "+str(j) +":" + df_c[str(j) +"_Link"].iloc[i])
      if(content==""):
        row_dict[str(j)] = "This is Empty Content"
        continue
      cont_l= content.split("\n")
      cont_l_1=[]
      for ii in cont_l:
        if ii == "":
          continue
        if len(ii.split(" "))>450:
          print("I am a problem, u shud see me", i,j)

        cont_l_1.append(ii)
      # cont_l_2 = combine_strs(cont_l_1)
      # cont_l_1=cont_l_2
      print(len(cont_l_1))
      len_array_1 = [len(i.split(" ")) for i in cont_l_1]
      len_array.extend(len_array_1)

      scores = []

      row_dict['query']=q
      for c,ii in enumerate(cont_l_1):
        score = score_for_one(q, [ii])
        scores.append((score, c))
      scores = sorted(scores,reverse=True)
      candidates = scores[:top_k]
      print([i[0] for i in candidates])
      candidates = [(j,i) for i,j in candidates]
      candidates = sorted(candidates)
      if(candidates[0][0] <-4.8):
        row_dict[str(j)] = "This is Useless Content"
        continue

      candidates=[cont_l_1[i[0]] for i in candidates]
      # candidates=[candidates[i[1]] for i in candidates]
      row_dict[str(j)] = " ".join(candidates)
      deli.deli_2()
      deli.deli_2()
      printw("Selected Content "+str(j)+ ": "+" ".join(candidates) )
      # row_dict[str(j)] = " ".join(candidates)
      deli.deli_2()
      input()
      output.clear()

    df_ans  = df_ans.append(row_dict,ignore_index=True)
  print(make_histogram(len_array))
  return df_ans

def rank_sentences(q, passage, n_cls=7, select_cls=4):
  sent_l = nltk.sent_tokenize(passage)
  print("Total sentences:",len(sent_l))
  if(len(sent_l)==0 or (len(sent_l)==1 and sent_l[0]=='')):
    return "Insufficient String"
  if(len(sent_l)<n_cls):
    return " ".join(passage)
  scores = []
  for i, k  in enumerate(sent_l):
    score = score_for_one(q,[k])
    scores.append((score,i))
  scores = sorted(scores,reverse=True)
  # print(scores)
  scoo = [i[0] for i in scores]
  x= np.array(scoo)
  km = KMeans(n_clusters=n_cls, random_state=42, max_iter=500)
  km.fit(x.reshape(-1,1))
  y = km.labels_
  sent_ll = [sent_l[iiii] for jjjj,iiii in scores]
  # print(list(zip(sent_ll,y)))
  candidates= []
  true_label = select_cls
  selecting = set()
  for ii,iii in enumerate(y):
    if(len(selecting)<select_cls):
      selecting.add(iii)
    if iii in selecting:
      candidates.append((scoo[ii], scores[ii][1]))
  print("Selected Sentences Count:",len(candidates))
  candidates = [(j,i) for i,j in candidates]
  candidates = sorted(candidates)
  candidates=[sent_l[i[0]] for i in candidates]
  return " ".join(candidates)


def rank_passages_2(df_c,top_k=9):
  df_c=df_c.fillna("")
  df_ans = pd.DataFrame()
  # len_array=[]
  for i, k in df_c.iterrows():
    row_dict={}
    for j in range(10):
      print("Index",i,j)
      passage_ranker=[]
      q=df_c['query'].iloc[i]

      content= df_c[str(j) +"_Content"].iloc[i]
      # printw("Question "+str(i) +":" + q)
      # printw("Content "+str(j) +":" + df_c[str(j) +"_Link"].iloc[i])
      if(content==" this is a meaningless content due to repetition and timeout as well"):
        row_dict[str(j)] =" this is a meaningless content due to repetition and timeout as well"
        continue
      start = timeit.timeit()
      if i == 58 and j ==6:
        row_dict[str(j)] = " this is a meaningless content due to repetition and timeout as well"
        continue
      row_dict[str(j)] = rank_sentences(q,content)
      end = timeit.timeit()
      print(end-start)

    df_ans  = df_ans.append(row_dict,ignore_index=True)
  # print(make_histogram(len_array))
  return df_ans

In [ ]:
df = pd.read_csv('extracted_contents_200.csv')
len(df)

200

In [ ]:
df.head()

In [ ]:
# df_sub = df.sample(n=10, random_state=42)
# df_sub = df_sub.reset_index()
startt = timeit.timeit()
df_c = rank_passages_3(df)
end  =  timeit.timeit()
print(end-startt)
print(df_c)


In [ ]:
(df_c).head(10)

In [ ]:
print(len(df_c))

150


In [ ]:
df_c.to_csv("dpr_200.csv",index=False)

In [ ]:
df_c = pd.read_csv('s.csv')

In [ ]:
df_c.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'query'], dtype='object')

In [ ]:
cont = '''

k-means clustering is a method of vector quantization, originally from signal processing, that aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid), serving as a prototype of the cluster. This results in a partitioning of the data space into Voronoi cells. k-means clustering minimizes within-cluster variances (squared Euclidean distances), but not regular Euclidean distances, which would be the more difficult Weber problem: the mean optimizes squared errors, whereas only the geometric median minimizes Euclidean distances. For instance, better Euclidean solutions can be found using k-medians and k-medoids.

The problem is computationally difficult (NP-hard); however, efficient heuristic algorithms converge quickly to a local optimum. These are usually similar to the expectation-maximization algorithm for mixtures of Gaussian distributions via an iterative refinement approach employed by both k-means and Gaussian mixture modeling. They both use cluster centers to model the data; however, k-means clustering tends to find clusters of comparable spatial extent, while the Gaussian mixture model allows clusters to have different shapes.

The unsupervised k-means algorithm has a loose relationship to the k-nearest neighbor classifier, a popular supervised machine learning technique for classification that is often confused with k-means due to the name. Applying the 1-nearest neighbor classifier to the cluster centers obtained by k-means classifies new data into the existing clusters. This is known as nearest centroid classifier or Rocchio algorithm.

The term "k-means" was first used by James MacQueen in 1967,[2] though the idea goes back to Hugo Steinhaus in 1956.[3] The standard algorithm was first proposed by Stuart Lloyd of Bell Labs in 1957 as a technique for pulse-code modulation, although it was not published as a journal article until 1982.[4] In 1965, Edward W. Forgy published essentially the same method, which is why it is sometimes referred to as the Lloyd–Forgy algorithm.[5]

Commonly used initialization methods are Forgy and Random Partition.[10] The Forgy method randomly chooses k observations from the dataset and uses these as the initial means. The Random Partition method first randomly assigns a cluster to each observation and then proceeds to the update step, thus computing the initial mean to be the centroid of the cluster's randomly assigned points. The Forgy method tends to spread the initial means out, while Random Partition places all of them close to the center of the data set. According to Hamerly et al.,[10] the Random Partition method is generally preferable for algorithms such as the k-harmonic means and fuzzy k-means. For expectation maximization and standard k-means algorithms, the Forgy method of initialization is preferable. A comprehensive study by Celebi et al.,[11] however, found that popular initialization methods such as Forgy, Random Partition, and Maximin often perform poorly, whereas Bradley and Fayyad's approach[12] performs "consistently" in "the best group" and k-means++ performs "generally well".


'''

query = "Is Kmeans computationally efficient?"
deli.deli_2()
ans = rank_sentences(query, cont, n_cls=7, select_cls =4)
print(ans)
print(get_answer_1(query,ans))

In [ ]:
df_c['query'] = df['query'].tolist()

In [ ]:
def rank_pages(df_c):
  df_c= df_c.fillna("Insufficient String")
  list_1= []
  for i, k in df_c.iterrows():
    print(i)
    q = k['query']
    temp_list = []
    for j in range(10):
      print("\t",j)
      try:
        c = k[str(j)]

        # print((c))
        try:
          # print("I am here")
          if(c == 'Insufficient String' or c ==' ' or c=='' ):
            continue
          print("\t\t",len(nltk.sent_tokenize(c)))
          ans, scores = get_answer_1(q,c)

          temp_list.append((harmonic_mean(scores), j, ans))
        except Exception as eeee:

          print("Error at stage 1:",eeee)

          c = rank_pass(q,c)
          print("\t\t After I have changed",len(nltk.sent_tokenize(c)))

          ans,scores = get_answer_1(q,c)
          temp_list.append((harmonic_mean(scores), j, ans))
      except:
        print("skipped bcz it took time")
    temp_list = sorted(temp_list, reverse=True)
    temp_list= temp_list[:3]
    # print(temp_list)
    temp_list = [ ii[2] for ii in temp_list]
    list_temp =[q]
    list_temp.extend(temp_list)
    list_1.append(list_temp)
  return list_1

In [ ]:
df_c = pd.read_csv('all_distilroberta_v1.csv')

In [ ]:
list_1 = rank_pages(df_c)



In [ ]:
import pickle
ff = open('200_all_distilroberta_v1_answers.txt','wb')
pickle.dump(list_1,ff)
ff.close()

In [ ]:
list_1

In [ ]:
list_1 = [['which is the most common use of opt-in e-mail marketing',
  'advertising',
  'to nurture your prospects through the sales process',
  'growing your email list',
  'flip the current card',
  ' which is the most common use of opt-in e-mail marketing C o p y t h i s t o m y a c c o u n t    E - m a i l t o a f r i e n d    F i n d o t h e r a c t i v i t i e s    S t a r t o v e r    H e l p          V i e w p r o f i l e'],
 ['how i.met your mother who is the mother',
  'Tracy McConnell',
  'Tracy McConnell',
  'Tracy McConnell',
  'She was the wife of Ted Mosby and The Mother of his children. The Mother\'s apartment was 317 West 115th Street, apartment 7A, until she moved in with Ted. After Tracy leaves, future Ted remarks, "Kids, that was the first drink your mother ever bought me". However, they postpone their wedding indefinitely when Tracy announces that she is pregnant. When Ted recalls how Robin\'s mother nearly didn\'t attend her wedding, Tracy makes an offhand comment about mothers always attending their daughter\'s weddings. Because if I hadn\'t, I never would\'ve met your mother." This strongly suggested that Ted meets The Mother in New York. Well, I met your mom." At the end of How I Met Everyone Else, Ted asks Marshall and Lily where his wife is, at his college reunion. Many speculated this was a clue that The Mother would be dead in the future and, even though that was the case, the timeline suggests his comment was simply made due to Ted being high. In Wait For It, it is revealed by Future Ted that The Mother\'s yellow umbrella factors into the story of how she and Ted met. In The Three Days Rule, Future Ted told his children that when he got their mother\'s phone number, he called her right away. Future Ted narrates how this was the best job he\'s ever had, and that if it weren\'t for that job, he never would have met their Mother. In Double Date, Future Ted told his kids that when he first told their mother his "shellfish-selfish" joke, she laughed. Cindy\'s roommate is The Mother, as revealed by Future Ted. In Big Days, Future Ted reveals that Ted will meet The Mother on the day of a wedding and that "a little ways down the road" he will be the best man at a wedding, later revealed in Challenge Accepted and The Magician\'s Code - Part Two to be Barney and Robin\'s wedding. In Garbage Island, in a scene that flashes forward to 2021, Ted meets Wendy the Waitress at an airport and tells her he is now married to "a wonderful woman" and has two kids, revealing that he will have met and married The Mother by this time. Later in The Exploding Meatball Sub, a scene set in April 2021 (ten years into the future) shows Ted wearing a wedding ring, further implying he will have met and married The Mother by this time. Barney complains about Ted bringing his baby daughter along, but relents because she is too cute. However, in season nine, it is revealed Ted and the Mother did not actually wed until 2020, after seven years together. Show creator Craig Thomas confirmed this was the place where Ted and The Mother would finally meet. In Bad Crazy, Future Ted reveals that the last girl he dated before meeting The Mother was Jeanette. In The Time Travelers, it is revealed that in April 2013, 45 days before she met Ted, The Mother was dating a man named Louis while living at 317 West 115th Street in apartment 7A and still obtaining her Economics degree. In How Your Mother Met Me, it is revealed The Mother\'s 21st birthday was in September 2005, this makes her birth date 1984, which makes her 28 by the events of Barney and Robin\'s wedding. It is revealed in the final episode of the series that the Mother had died six years prior to when Ted began telling his children their story. This is a list of identities and popular theories regarding The Mother, before she was revealed:     At the end of Belly Full of Turkey, Future Ted jokes to his kids that the stripper named Tracy he flirts with was their mother. Their shocked reaction when he tells them "And that, kids, is the true story of how I met your mother." The mother\'s name was confirmed as "Tracy McConnell',
  'Lily Aldrin'],
 ['what type of fertilisation takes place in humans',
  'External fertilization',
  "A pregnancy starts with fertilization, when a woman's egg joins with a man's sperm. Fertilization usually takes place in a fallopian tube that links an ovary to the uterus. If the fertilized egg successfully travels down the fallopian tube and implants in the uterus, an embryo starts growing. When a fertilized egg does implant, a hormone called human chorionic gonadotropin (hCG) begins to be produced in the uterus.",
  "A pregnancy starts with fertilization, when a woman's egg joins with a man's sperm. Fertilization usually takes place in a fallopian tube that links an ovary to the uterus. If the fertilized egg successfully travels down the fallopian tube and implants in the uterus, an embryo starts growing. When a fertilized egg does implant, a hormone called human chorionic gonadotropin (hCG) begins to be produced in the uterus.",
  'External fertilization',
  'Human fertilization is the union of a human egg and sperm, occurring in the ampulla of the fallopian tube. Scientists discovered the dynamics of human fertilization in the nineteenth century. [2]     The process of fertilization involves a sperm fusing with an ovum. Various exceptions to this sequence are possible, including artificial insemination, in vitro fertilization, external ejaculation without copulation, or copulation shortly after ovulation. In vitro fertilization (IVF) is a process by which egg cells are fertilized by sperm outside the womb, in vitro.'],
 ['who had the most wins in the nfl',
  'The New England Patriots',
  'Chicago Bears',
  'The New England Patriots',
  'the Packers',
  'New England Patriots'],
 ['who played mantis guardians of the galaxy 2',
  'Klementieff',
  'Klementieff',
  'Klementieff',
  'Klementieff',
  'Klementieff'],
 ['what indian tribe did the acadians form friendships and alliances with',
  "Mi'kmaq",
  "Mi'kmaq",
  "Mi'kmaq",
  "Mi'kmaq",
  "Mi'kmaq"],
 ['who is deputy cm of j and k',
  'Kavinder Gupta',
  'Kavinder Gupta',
  'Kavinder Gupta',
  'Gupta',
  'Kavinder Gupta'],
 ['the nashville sound brought a polished and cosmopolitan sound to country music by',
  'the use of lush string arrangements with a real orchestra and often background vocals provided by a choir',
  'the use of lush string arrangements with a real orchestra and often background vocals provided by a choir',
  'the use of lush string arrangements with a real orchestra and often background vocals provided by a choir',
  'Eddy Arnold',
  'Owen Bradley'],
 ['what channel is the premier league on in france',
  'ABCATA',
  'Includes',
  'YouTube',
  'YouTube',
  'YouTube'],
 ['what is the use of jdk in java',
  'a development environment for building applications, applets, and components using the Java programming language',
  'developing Java applications and applets',
  'The Java Development Kit (JDK) is a cross-platformed software development environment that offers a collection of tools and libraries necessary for developing Java-based software applications and applets',
  'to convert your source code into a format that the Java Runtime Environment (JRE) can execute',
  'allows developers to create Java programs that can be executed and run by the JVM and JRE'],
 ['who needs to be in the car with a permit driver',
  '21 or older',
  "States' graduated driving laws differ on who can be in the car. Typically, drivers with learner  s permits can  t drive alone. A licensed driver at least 21 years old",
  'a licensed adult',
  'a licensed adult age 21 or older',
  'Young adults in all states now need to go through a graduated driver  s license (GDL) program that encourages teen driving safety. Applicants getting their permit must meet minimum age requirements; typically, teen drivers are between 14 and 16 years old'],
 ["god's not dead a light in the darkness release date",
  'March 30, 2018',
  'March 30, 2018',
  'March 30, 2018',
  'March 30, 2018',
  'August 21, 2018'],
 ['who is the current president of un general assembly',
  'Abdulla Shahid',
  'Abdulla Shahid',
  'María Fernanda Espinosa',
  'Joseph Deiss',
  'Mr. John W. Ashe'],
 ['when do they pull the powerball numbers 2016',
  'every Wednesday and Saturday at 10:59 p.m. Eastern time',
  'May 7, 2016',
  'August 23, 2021',
  'Wednesday, March 2, 2016',
  'August 23, 2021'],
 ['when do the eclipse supposed to take place',
  'Monday, April 8, 2024',
  'at the phase of new moon, when the moon passes directly between the sun and Earth and its shadows fall upon Earth  s surface',
  'December 4, 2021',
  'June 10',
  'June 10, 2021'],
 ['what is the name of the sea surrounding dubai',
  'Persian Gulf',
  'Persian Gulf',
  'Persian Gulf',
  'Persian Gulf',
  'Persian'],
 ['who holds the nba record for most points in a career',
  'Abdul-Jabbar',
  'Kareem Abdul-Jabbar',
  'Chambers',
  "2,027-point season in 1983  84, earning the first of his back-to-back All-NBA First Team selections. Over two years, he averaged 21.9 points per game, in his first year playing alongside players such as 1980 NBA All-Star World B. He scored 50 points on 20 for 30 shooting with 10 free throws in a 117  113 Knicks' victory over the San Antonio Spurs on January 31. King shot 20 for 28 from the field with 10 free throws in the 105  98 win over the Mavericks. [3][4]     The next season, on Christmas Day, 1984, King lit up the New Jersey Nets for 60 points in a losing effort, becoming just the tenth player in NBA history to score 60 or more points in a single game. To that point no NBA player had returned to form after such a potentially career-ending injury, surgery, and loss of time. Despite averaging 22.7 points per game during his first six games back, he had not recovered his pre-injury explosiveness and was released by the Knicks at the end of the 1987 season. [7]     King retired with 19,655 points in 874 games, good for a 22.5 points per game average and number 16 on the all-time NBA scoring list at the time of his retirement. [13][15]     List of National Basketball Association career scoring leaders   List of National Basketball Association career turnovers leaders   List of individual National Basketball Association scoring leaders by season   List of National Basketball Association top rookie scoring averages   List of National Basketball Association players with most points in a game   ^ a b Lawrence, Mitch",
  'Dennis Rodman'],
 ['once upon a time season 6 episode list', '', '', '', '', ''],
 ['where is a georgia concealed carry permit valid',
  'on public college campuses',
  'Wyoming',
  'The AZ concealed weapons permit (CWP) is honored by GA through a reciprocal agreement, while GA does not require a CCW (GA right to carry/open carry) for loaded possession in vehicles, while fishing, hunting, home, business. The GA law says you must have a CCW to openly carry. The AZ concealed weapons permit (CWP) is honored by GA through a reciprocal agreement, while GA does not require a CCW (GA right to carry/open carry) for loaded possession in vehicles, while fishing, hunting, home, business. The GA law says you must have a CCW to openly carry.',
  'residents of Georgia',
  'In parks, historic sites, and recreational areas;1   Within 150 feet of any polling place although an exception may apply for concealed carry permittees entering government buildings when they are open for business and where ingress into such buildings is not restricted or screened by security personnel. These location restrictions do not apply to a firearm possessed by a license holder if the firearm is under the license holder  s control in a motor vehicle or is in a locked compartment, container, or firearms rack in or on a motor vehicle parked in parking facility.4 A person with a valid license may also carry a handgun in parks, historic sites, and recreational areas, including in all publicly owned buildings located in such areas, and in wildlife management areas, and on public transportation, except where prohibited by federal law.5 Georgia also authorizes in airport terminals and aircraft, except where prohibited by federal law.6 It authorizes the governing body or authority of a place of worship to permit the carrying of weapons or long guns by license holders.7. And it authorizes a license holder to carry a weapon in a government building if the building is open for business and   where ingress into such building is not restricted or screened by security personnel.  8   A law'],
 ['when did the new maze runner movie come out',
  'January 26, 2018',
  'January 26, 2018',
  'January 26, 2018',
  'January 26, 2018',
  'January 26, 2018'],
 ['how many players on a box lacrosse team', '19', 'six', 'six', 'six', 'six'],
 ['where are the upcoming olympics to be held',
  'Beijing',
  'Beijing',
  'Beijing',
  'Tokyo',
  'Beijing'],
 ['when did the nba 3 second rule start',
  '2001-02 season',
  '2001 season',
  '1936',
  'to tame the impact of the newly legalized zone defense rules',
  '2015-16'],
 ['when were the plus and minus signs first recorded',
  '1481',
  '1494',
  'the late 15th century',
  'Robert Recorde  s book,  The Whetstone of Witte',
  '2002-12-12'],
 ["where does the phrase bob's your uncle come from",
  'The origins are uncertain, but a common theory is that the expression arose after Conservative Prime Minister Robert Cecil known as Bob appointed his nephew Arthur Balfour as Chief Secretary for Ireland in 1887, an act of favoritism which was apparently both surprising and unpopular. Whatever other qualifications Balfour might have had,   Bob  s your uncle   was seen as the main one.',
  'The origins are uncertain, but a common theory is that the expression arose after Conservative Prime Minister Robert Gascoyne-Cecil, 3rd Marquess of Salisbury ("Bob") appointed his nephew Arthur Balfour as Chief Secretary for Ireland in 1887, an act of nepotism, which was apparently both surprising and unpopular. Whatever other qualifications Balfour might have had, "Bob\'s your uncle" was seen as the conclusive one.',
  'The origins are uncertain, but a common theory is that the expression arose after Conservative Prime Minister Robert Gascoyne-Cecil, 3rd Marquess of Salisbury ("Bob") appointed his nephew Arthur Balfour as Chief Secretary for Ireland in 1887, an act of nepotism, which was apparently both surprising and unpopular. Whatever other qualifications Balfour might have had, "Bob\'s your uncle" was seen as the conclusive one.',
  'The origins are uncertain, but a common theory is that the expression arose after Conservative Prime Minister Robert Gascoyne-Cecil, 3rd Marquess of Salisbury ("Bob") appointed his nephew Arthur Balfour as Chief Secretary for Ireland in 1887, an act of nepotism, which was apparently both surprising and unpopular. Whatever other qualifications Balfour might have had, "Bob\'s your uncle" was seen as the conclusive one.',
  'The origins are uncertain, but a common theory is that the expression arose after Conservative Prime Minister Robert Gascoyne-Cecil, 3rd Marquess of Salisbury ("Bob") appointed his nephew Arthur Balfour as Chief Secretary for Ireland in 1887, an act of nepotism, which was apparently both surprising and unpopular. Whatever other qualifications Balfour might have had, "Bob\'s your uncle" was seen as the conclusive one.'],
 ['the one where chandler and monica get engaged',
  '26',
  "When Chandler gets home, Joey stops him in the hallway and tells him Monica left; that she'd packed a bag and was going to stay with her parents. Chandler becomes depressed, thinking he screwed everything up, but when he enters the apartment, he sees that Monica is there and has decorated the whole apartment with candles. Monica gets down on one knee and tries to propose to Chandler, but she starts crying so much that she can't finish it.",
  "When Chandler gets home, Joey stops him in the hallway and tells him Monica left; that she'd packed a bag and was going to stay with her parents. Chandler becomes depressed, thinking he screwed everything up, but when he enters the apartment, he sees that Monica is there and has decorated the whole apartment with candles. Monica gets down on one knee and tries to propose to Chandler, but she starts crying so much that she can't finish it.",
  "When Chandler gets home, Joey stops him in the hallway and tells him Monica left; that she'd packed a bag and was going to stay with her parents. Chandler becomes depressed, thinking he screwed everything up, but when he enters the apartment, he sees that Monica is there and has decorated the whole apartment with candles. Monica gets down on one knee and tries to propose to Chandler, but she starts crying so much that she can't finish it.",
  "When Chandler gets home, Joey stops him in the hallway and tells him Monica left; that she'd packed a bag and was going to stay with her parents. Chandler becomes depressed, thinking he screwed everything up, but when he enters the apartment, he sees that Monica is there and has decorated the whole apartment with candles. Monica gets down on one knee and tries to propose to Chandler, but she starts crying so much that she can't finish it."],
 ['who won so you think you can dance 2016',
  'Bailey Muñoz',
  'Kida',
  'Kida',
  'Kida',
  'Kida'],
 ['who plays norman bates in the tv show',
  'Freddie Highmore',
  'Freddie Highmore',
  'Freddie Highmore',
  'Freddie Highmore',
  'Freddie Highmore'],
 ['when did usa start driving on the right',
  '1792',
  'late 1700s',
  'When cars were invented, Henry Ford was a big influence in American driving customs as he built his Model T with the driver on the left side of the car, meaning drivers would have to drive on the right side of the road so their passengers could exit the car on the curb and not into oncoming traffic. Most countries followed America  s lead, such as Canada, Italy and Spain in the 1920  s, eastern Europe in 1930  s and Scandinavia in the 1960  s.',
  '1908',
  'S o m e t h i n g w e n t w r o n g. W a i t a m o m e n t a n d t r y a g a i n.'],
 ['name some components of the central nervous system (cns)',
  'brain',
  'brain',
  'brain',
  'brain',
  'brain'],
 ["who's the director of the price is right",
  'Adam Sandler',
  'Adam Sandler',
  'Adam Sandler',
  'Adam Sandler',
  'Adam Sandler'],
 ['bridge on the river kwai fact or fiction',
  'largely fictional',
  'based on the 1952 novel written by Pierre Boulle.',
  'based on the 1952 novel written by Pierre Boulle.',
  'based on the 1952 novel written by Pierre Boulle.',
  'based on the 1952 novel written by Pierre Boulle.'],
 ['is hong kong a part of the commonwealth',
  "The transfer of sovereignty over Hong Kong in 1997 ended the territory's status as a part of the Commonwealth through the United Kingdom. Hong Kong has nevertheless continued to participate in some of the organisations of the Commonwealth family, such as the Commonwealth Lawyers Association (hosted the Commonwealth Lawyers Conference in 1983 and 2009), the Commonwealth Parliamentary Association (and the Westminster Seminar on Parliamentary Practice and Procedures), the Association of Commonwealth Universities and the Commonwealth Association of Legislative Counsel,[111][112] as well as the Commonwealth War Graves Commission (CWGC).",
  'Some countries and regions could also join the Commonwealth on the basis of having a constitutional historical link to an existing member of being part of the British Empire including: Afghanistan, Hong Kong, Iraq, Jordan, Kuwait, Myanmar, Oman, Qatar, Sudan, the United Arab Emirates, and the United States.',
  'Some countries and regions could also join the Commonwealth on the basis of having a constitutional historical link to an existing member of being part of the British Empire including: Afghanistan, Hong Kong, Iraq, Jordan, Kuwait, Myanmar, Oman, Qatar, Sudan, the United Arab Emirates, and the United States.',
  'Some countries and regions could also join the Commonwealth on the basis of having a constitutional historical link to an existing member of being part of the British Empire including: Afghanistan, Hong Kong, Iraq, Jordan, Kuwait, Myanmar, Oman, Qatar, Sudan, the United Arab Emirates, and the United States.',
  'Some countries and regions could also join the Commonwealth on the basis of having a constitutional historical link to an existing member of being part of the British Empire including: Afghanistan, Hong Kong, Iraq, Jordan, Kuwait, Myanmar, Oman, Qatar, Sudan, the United Arab Emirates, and the United States.'],
 ['where can pulse be felt in the body',
  'over an artery that lies near the surface of the skin',
  'The best places to take your pulse are at your wrist',
  'wrist',
  'S o m e t h i n g w e n t w r o n g. W a i t a m o m e n t a n d t r y a g a i n.',
  'S o m e t h i n g w e n t w r o n g. W a i t a m o m e n t a n d t r y a g a i n.'],
 ['why did argentina attack the falklands/malvinas islands',
  'to capture the Falkland Islands',
  'On 2 April 1982, Argentinian forces invaded the British overseas territory of the Falkland Islands. Argentina had claimed sovereignty over the islands for many years and their ruling military junta did not believe that Britain would attempt to regain the islands by force.',
  '',
  'as a means of promoting patriotic feeling and propping up its regime',
  "Argentina's original intention was to mount a quick, symbolic occupation, followed rapidly by a withdrawal, leaving only a small garrison to support the new military governor. Argentine assault units were indeed withdrawn to the mainland in the days following the invasion, but strong popular support and the rapid British reaction forced the Junta to change their objectives and reinforce the islands, since they could not politically afford to lose the islands once the British came out to fight. Admiral Lombardo feared that Operation Alpha would jeopardise the secret preparations for the Falkland landings, but Admiral Otero had close links to Admiral Anaya who approved Operation Alpha despite promising Admiral Lombardo he would cancel the operation. The Falkland Islands Governor and subsequently the Foreign Office passed a message back to the BAS team for passing to the captain of the ARA Bahia Buen Suceso. This was to the effect that the Argentine flag must be taken down and that they must report to the British administrator (Mr Stephen Martin, commander of the British Antarctic Survey Base) at Grytviken, to have their passports stamped (which they refused to do as it would acknowledge British sovereignty over the islands). The full party of Royal Marines was not landed until 31 March when it became apparent that Argentine forces intended to seize the Falkland Islands"],
 ['how long does it take for the moon to go around earth',
  '29.5 days',
  'about one month',
  'approximately 27 days',
  "about 27.32 days (a tropical month and sidereal month) and one revolution relative to the Sun in about 29.53 days (a synodic month). Earth and the Moon orbit about their barycentre (common center of mass), which lies about 4,670  km (2,900  mi) from Earth's center (about 73% of its radius), forming a satellite system called the Earth  Moon system. On average, the distance to the Moon is about 385,000  km (239,000  mi) from Earth's center, which corresponds to about 60 Earth radii or 1.282 light-seconds. With a mean orbital velocity of 1.022  km/s (0.635 miles/s),[9] the Moon covers a distance approximately its diameter, or about half a degree on the celestial sphere, each hour. The Moon's orbit around Earth has many variations (perturbations) due to the gravitational attraction of the Sun and planets, the study of which (lunar theory) has a long history. [10]     The orbit of the Moon is a nearly circular ellipse about the Earth (the semimajor and semiminor axes are 384,400  km and 383,800  km, respectively: a difference of only 0.16%). The lunar orbit's major axis    the longest diameter of the orbit, joining its nearest and farthest points, the perigee and apogee, respectively    makes one complete revolution every 8.85 Earth years, or 3,232.6054 days",
  "about 27.32 days (a tropical month and sidereal month) and one revolution relative to the Sun in about 29.53 days (a synodic month). Earth and the Moon orbit about their barycentre (common center of mass), which lies about 4,670  km (2,900  mi) from Earth's center (about 73% of its radius), forming a satellite system called the Earth  Moon system. On average, the distance to the Moon is about 385,000  km (239,000  mi) from Earth's center, which corresponds to about 60 Earth radii or 1.282 light-seconds. With a mean orbital velocity of 1.022  km/s (0.635 miles/s),[9] the Moon covers a distance approximately its diameter, or about half a degree on the celestial sphere, each hour. The Moon's orbit around Earth has many variations (perturbations) due to the gravitational attraction of the Sun and planets, the study of which (lunar theory) has a long history. [10]     The orbit of the Moon is a nearly circular ellipse about the Earth (the semimajor and semiminor axes are 384,400  km and 383,800  km, respectively: a difference of only 0.16%). The lunar orbit's major axis    the longest diameter of the orbit, joining its nearest and farthest points, the perigee and apogee, respectively    makes one complete revolution every 8.85 Earth years, or 3,232.6054 days"],
 ['who is the prime minister of republic of mauritius',
  'Pravind Jugnauth',
  'Pravind Jugnauth',
  'Pravind Jugnauth',
  'Pravind Jugnauth',
  'Pravind Jugnauth'],
 ['what teams are in the fa cup final',
  'Chelsea',
  'Chelsea',
  'Chelsea',
  'Leicester City',
  'Leicester City'],
 ["where is a midsummer night's dream set",
  'partly in the city of Athens',
  'the court of Athens and the woods',
  'ancient Athens and the wood just outside it',
  'Athens, Greece',
  'ancient Athens'],
 ['when does the new series of black sails start',
  'January 29, 2017',
  'January 29, 2017',
  'When does Black Sails season 5 come out? Starz has officially cancelled Black Sails tv series. Season 4 of Black Sails will be the last season. Was Black Sails renewed for season 5? No, Black Sails has been cancelled by Starz. Is Black Sails cancelled? Yes, Black Sails has been officially canceled.',
  'Although Starz renewed  Black Sails  for season four, back in the summer of 2015, by  July of 2016, the premium cabler issued a press release announcing the  renewal was for the the pirate drama  s fourth and final season.',
  'June 21, 2019'],
 ['who is the first person who went to moon',
  'Neil Armstrong',
  'Neil Armstrong',
  'Apollo 11 (July 16  24, 1969) was the spaceflight that first landed humans on the Moon. Armstrong',
  'Neil Armstrong',
  'Neil Armstrong'],
 ['when did the first lego movie come out',
  '2014',
  'February 7, 2014',
  '2019',
  'September 2016',
  '2014'],
 ['are pure metals made of atoms or ions',
  'atoms',
  'atoms',
  'atoms',
  'ions',
  ''],
 ['what episode in victorious is give it up',
  'Freak the Freak Out',
  "The episode ends where Jade and Beck are officially a couple again and Robbie finally gets the butterfly out of Cat's ear. Robbie gives in and sells Rex, but ends up regretting his decision.",
  "Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story but you never get it right  Give it up  Come a little closer  Come a little closer baby, baby  Come a little closer  Come a little closer baby, baby  Liz:  So stop trying to walk away  No, you won't ever leave me behind  Ariana:  Nooooo  You better believe  That I'm here to stay (Liz: That's right)  'Cause you're the shade  And I'm the sunshine  Ooooooh  Both:  Look at me, boy  'Cause I got you where I want you  Isn't it so exciting? Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story but you never get it right  Give it up  [At Tori and Trina's house]  Tori: Okay, it says that you're supposed to take one of these antibiotics every six hours. Wanna shake you, wanna break you  Take the backseat, boy,  'Cause now I'm driving  Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story  But you never get it right  Give it up, ohh whoa, yeah        ",
  "Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story but you never get it right  Give it up  Come a little closer  Come a little closer baby, baby  Come a little closer  Come a little closer baby, baby  Liz:  So stop trying to walk away  No, you won't ever leave me behind  Ariana:  Nooooo  You better believe  That I'm here to stay (Liz: That's right)  'Cause you're the shade  And I'm the sunshine  Ooooooh  Both:  Look at me, boy  'Cause I got you where I want you  Isn't it so exciting? Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story but you never get it right  Give it up  [At Tori and Trina's house]  Tori: Okay, it says that you're supposed to take one of these antibiotics every six hours. Wanna shake you, wanna break you  Take the backseat, boy,  'Cause now I'm driving  Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story  But you never get it right  Give it up, ohh whoa, yeah        ",
  "Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story but you never get it right  Give it up  Come a little closer  Come a little closer baby, baby  Come a little closer  Come a little closer baby, baby  Liz:  So stop trying to walk away  No, you won't ever leave me behind  Ariana:  Nooooo  You better believe  That I'm here to stay (Liz: That's right)  'Cause you're the shade  And I'm the sunshine  Ooooooh  Both:  Look at me, boy  'Cause I got you where I want you  Isn't it so exciting? Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story but you never get it right  Give it up  [At Tori and Trina's house]  Tori: Okay, it says that you're supposed to take one of these antibiotics every six hours. Wanna shake you, wanna break you  Take the backseat, boy,  'Cause now I'm driving  Give it up, you can't win  'Cause I know where you been  Such a shame you don't put up a fight  That's a game that we play at the end of the night  It's the same old story  But you never get it right  Give it up, ohh whoa, yeah        "],
 ['movement of molecules across a membrane powered by atp',
  'Primary active transport moves ions across a membrane and creates a difference in charge across that membrane. The primary active transport system uses ATP to move a substance, such as an ion, into the cell, and often at the same time, a second substance is moved out of the cell. The sodium-potassium pump move potassium and sodium ions across the plasma membrane. ATP itself is formed through secondary active transport using a hydrogen ion gradient in the mitochondrion. The pocket pinches off, resulting in the particle being contained in a newly created vacuole that is formed from the plasma membrane.',
  'active transport',
  'Primary active transport',
  'Active transport powered by adenosine triphosphate (ATP) is known as primary active transport',
  'Active transport'],
 ['who sold manhattan to the dutch in 1626',
  'Peter Minuit',
  'Peter Minuit',
  'Peter Minuit',
  'Peter Minuit',
  'Peter Minuit'],
 ['city of stars are you shining just for me meaning',
  'a love letter to a place (Los Angeles), to a person (Sebastian to Mia, and Mia to Sebastian), and to love itself.',
  'The A sections of   City of Stars   focus on the difficulty of forging connections in the modern metropolis of Los Angeles. There  s a certain deliberate ambiguity as to whether Sebastian and Mia are singing about each other, or about their careers:   City of stars / Are you shining just for me?   Is that a plea for love in a lonely place, or a call for individual recognition from an unforgiving city and its creative industries? The meaning becomes clearer in the song  s contrasting B section, as the piano accompaniment moves to a bright, major-key harmony and the lyrics express a romantic yearning in crystalline terms:   A look in somebody  s eyes / To light up the skies / To open the world and send me reeling.     ',
  '"City of Stars" is a song performed by Ryan Gosling and Emma Stone in the film La La Land (2016). [...] I was just composing it from an emotional place and thinking about the tone. I would say the tone is hopeful, but melancholy at the same time. I was thinking about that idea a little bit and just trying to compose a melody that I thought was shapely and beautiful.',
  'In my opinion, it  s just a wonderful film. Songs from the movie are stuck in memory almost immediately, especially the main theme City of Stars. Setting is the old-fashioned, almost dreamlike Los Angeles or La La Land as it called by the locals. Every detail on the screen has its own bright history in the lenses of cameras.',
  'We personalize the universe and think that it is all happening for us. The city is busy with people who are desperate for love, though they dull their emotions through drinking and eating at the   in   places, they all are like little children, looking for love and affection, warmth and closeness. Because no one will ever be able to do that for you. It feels so good to know that someone cares and also excites you. It is wonderful and miraculous! Wonderful    This melts our hearts  !!'],
 ['who have been the winners of ink master',
  'Bubba Irwin',
  'Laura Marie Wachholder',
  'Dave Navarro   Chris Núñez   Oliver Peck   Matt Knight (season 1)   Rick Robles (seasons 2  13)   Steven Weinstock   Glenda Hersh   Andrea Richter   40  83 minutes   Truly Original   Spike (2012  18)   Paramount Network (2018  20)   Paramount+ (revival)   Ink Master: Redemption   Ink Master: Angels   Ink Master: Grudge Match   ',
  'Shane O  Neill',
  'Jason Clay Dunn'],
 ['what year did the movie deuces come out',
  'April 1, 2017',
  'April Fools Day',
  'Apr 18, 2017',
  '',
  ' what year did the movie deuces come out e r r o r c o d e : 1 0 2 0'],
 ['who is the present sport minister of india',
  'Anurag Thakur',
  'Anurag Thakur',
  'Anurag Thakur',
  'S',
  'S'],
 ['who has the most followers on instagram in the world',
  'National Geographic',
  'Cristiano Ronaldo',
  'Portuguese footballer Cristiano Ronaldo',
  'Instagram',
  'Khaby Lame'],
 ['where was life or something like it filmed',
  'Vancouver',
  'New York, USA',
  'Seattle television station, Lanie thinks her life is perfect. But her perfect world starts unraveling after a homeless street seer (Tony Shalhoub) tells Lanie that she leads a meaningless existence, and will die the following week. From Regency Enterprises, Life or Something Like It (2002) is directed by Stephen Herek. Ambitious, glamorous and a top reporter at the That  s Seattle',
  '',
  ''],
 ['the highest post in police department of karnataka state is',
  'Additional Director General of Police',
  'the senior most IPS officer in the intelligence bureau',
  'the senior most IPS officer in the intelligence bureau',
  'the senior most IPS officer in the intelligence bureau',
  'the senior most IPS officer in the intelligence bureau'],
 ['is it a bank holiday today in spain',
  'NEXT year people in Spain will be treated to eight national Bank Holidays in addition to the holidays decided by each autonomous community. Full-time workers in Spain are entitled to 22 days paid leave a year, and some employers choose to include Bank Holidays. You can take all your holidays at once or divide it up, but Spain  s labor laws require that at least one period of holiday leave must be two weeks long. In 2022  there are eight national holidays for everyone in Spain and there is also an additional two other holidays that fall on a Sunday.',
  'The government in Spain sets the Spanish national holidays each year, although Spanish bank holidays differ between each of the regions. While each city observes the national holidays in Spain, regional holidays are set by the local governments.',
  "New Year's Day is a public holiday in all countries that observe the Gregorian calendar,...     Epiphany  Jan 6,. Andalucía became an autonomous community of Spain on February 28th 1980     Public holidays are taken on the day they fall and not moved to the following weekend, if they fall on a Saturday or Sunday.",
  'Public holidays celebrated in Spain include a mix of religious (Roman Catholic), national and regional observances. If one of the "national holidays" happens to fall on a Sunday the regional governments    the autonomous communities of Spain    can choose an alternate holiday or they can allow local authorities to choose. Since 2010, Ceuta and Melilla, both autonomous cities of Spain, have declared the Muslim holiday of Eid al-Adha or Feast of the Sacrifice, as an official public holiday.',
  'Note: May be replaced by the autonomous communities with another date.'],
 ['who played the bank robber in dirty harry',
  'Albert Popwell',
  'Albert Popwell',
  'Albert Popwell',
  'Starred in the first four "Dirty Harry" movies with Clint Eastwood. He portrayed four different characters in each of the "Dirty Harry" films. He was the bank robber in Dirty Harry (1971), a pimp in Magnum Force (1973), a black militant leader in The Enforcer (1976) and Harry\'s partner in Sudden Impact (1983). Albert Popwell',
  'Popwell was featured on many television series, but is perhaps best known for his appearances in films opposite Clint Eastwood, whom he acted with in five films, starting with Coogan\'s Bluff (film) and in the first four films of the Dirty Harry series. Popwell was a wounded bank robber at the receiving end of Eastwood\'s "Do you feel lucky?" monologue from Dirty Harry.'],
 ['what is the worth of the catholic church',
  'more than $30 billion',
  "Bankers' best guesses about the  Vatican's wealth put it at $10 billion to $15 billion",
  '$6 billion',
  '',
  ''],
 ['where is the best farming soil in the us and where did all the dirt come from',
  "North America is blessed with a disproportionate share of the world's best agricultural soil. Our most essential and undervalued resource, soil is where the living world of biology meets the dead world of geology. The world's best agricultural soils are those that developed along with the native forests (ultisols and alfisols) and grasslands (mollisols) that long covered continents in the temperate regions. In these locations the addition of organic matter from leaf fall in forests or root growth in grasslands built up fertile soils with high agricultural capacity. And the best of these soils for agriculture are those developed on deposits of loess, windblown silt such as that covering much of the American Midwest. With a high proportion of finely ground, fresh mineral grains, soils developed on loess are renowned for their fertility. The same can't be said for soils in most of the world, where rock lies just one to several feet below ground.",
  "Farming has destroyed a lot of the rich soil of America's Midwestern prairie. The most fertile topsoil is entirely gone from a third of all the land devoted to growing crops across the upper Midwest",
  "The   Great Plains' of North America and the   Bread Basket' of Europe are some of the world's most productive agricultural soils because they developed under grassland vegetation, whose root biomass and decomposition resulted in SOM accumulation.",
  'The  mollisols',
  'Conrad  s soil among the richest farmland in the world.'],
 ['the pair of hand drums used in indian classical music is called',
  'The Tabla',
  'tabla',
  'tabla',
  'The chenda',
  'P'],
 ['where does the term petty officer come from',
  'the old French word, petit meaning something small',
  'Since entering the Coast Guard, I have chafed at the term   petty officer.   As a prior enlisted U.S. Army Air Cavalry member, I took pride when I was addressed as PFC or Specialist Moore, titles that eloquently  included me in the fighting unit. So I was taken aback when commissioned into the Coast Guard that I now was to call an experienced enlisted Coast Guardsman by the title of petty officer.',
  'the Age of Sail',
  'In medieval and later England just about every village had several "petite", "pety" or "petty" officials/officers who were subordinate to such major officials as the steward of sheriff. The petty officers were the assistants to the senior officials. Since the early seamen knew petty officers in their home villages they used the term to describe the minor officials aboard their ships. A ship\'s Captain or Master chose his own Petty Officers who served at his pleasure. At the end of a voyage or whenever the ship\'s crew was paid off and released the Petty Officers lost their positions and titles.',
  'The full appellation "Petty Officer 1st-Class" or "PO1" in speech is generally used only when the "first-class" distinction be made, such as to distinguish between members with similar names but differing ranks, or on promotion parades. It is common practice to refer to the Petty Officer by this shorthand in all but the most formal correspondence (such as printing and inscription on awards). Often, the Petty Officer is just referred to by the shorthand designation, without using the surname.'],
 ['what do the colors of the olympic rings mean',
  "the colours composing every competing country's flag at the time",
  "the colours composing every competing country's flag at the time",
  "the colours composing every competing country's flag at the time",
  "the colours composing every competing country's flag at the time",
  "the colours composing every competing country's flag at the time"]]

`rank_pages(df_c[:10])`

In [ ]:
list_1= []
df1=df1.fillna("")
for i in range(10):
  q = df1['query'].iloc[i]
  print(i)
  print("Question: ", q)
  print("Actual Answer:" ,df['ans'].iloc[i])
  for j in range(10):
    p = df1[str(j)].iloc[i]
    try:
      printw("Answer"+str(j)+": "+str(get_answer_1(q,p)))
    except:
      print(len(p.split(" ")))
      print("document shortening stricter required.")

#Trials and Runs


In [ ]:
df = pd.read_csv('content_link.csv')

In [ ]:
import pickle
data = pickle.load(open('150_start.txt','rb'))
q_l = [i[0] for i in data]
a_l = [i[1] for i in data]

In [ ]:
df1 = rank_passages_1(df_sub)

In [ ]:
f0 = open("ans_2_0_1.txt",'r')
f1 = open("ans_2_1_1.txt",'r')
f2 = open("ans_2_2_1.txt",'r')
f3 = open("ans_2_3_1.txt",'r')
f4 = open("ans_2_4_1.txt",'r')
f = open("gold_ans_2.txt",'r')
list_2=[]

for i in range(150):
  list_2_temp=[]
  list_2_temp.append(f.readline().strip())
  list_2_temp.append(f0.readline().strip())
  list_2_temp.append(f1.readline().strip())
  list_2_temp.append(f2.readline().strip())
  list_2_temp.append(f3.readline().strip())
  list_2_temp.append(f4.readline().strip())
  list_2.append(list_2_temp)


f0.close()
f1.close()
f2.close()
f3.close()
f4.close()
f.close()

In [ ]:
from google.colab import output
for i,k in df.iterrows():
  deli.deli_3()
  for j in range(10):
    print(k['query'])
    print(k[str(j)+"_Link"])
    print(k[str(j)+"_Content"])
    input()
    output.clear()

In [ ]:


def rank_pages_s(df_c):
  df_c= df_c.fillna("Insufficient String")
  list_1= []
  for i, k in df_c.iterrows():
    print(i)
    q = k['query']
    temp_list = []
    for j in range(10):
      try:
        c = k[str(j)]

        # print((c))
        try:
          print("I am here")
          if(c == 'Insufficient String' or c ==' ' or c=='' ):
            continue
          print(len(nltk.sent_tokenize(c)))
          ans, scores = get_answer_1(q,c)

          temp_list.append((harmonic_mean(scores), j, ans))
        except Exception as eeee:
          print("Error at stage 1:",eeee)
          c = rank_sentences(q,c)
          ans,scores = get_answer_1(q,c)
          temp_list.append((harmonic_mean(scores), j, ans))
      except:
        print("skipped bcz it took time")
    temp_list = sorted(temp_list, reverse=True)
    temp_list= temp_list[:3]
    # print(temp_list)
    temp_list = [ ii[2] for ii in temp_list]
    list_temp =[q]
    list_temp.extend(temp_list)
    list_1.append(list_temp)
  return list_1

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
import torch

import pickle

temp = pickle.load(open('new_embeddings.pkl','rb'))

vocab_size = len(temp)
embedding =  nn.Embedding(vocab_size, 100)
embed_list =[]
for word in trainDS.vocab._id2word:
  embed_list.append(temp[word])
weight_matrix= np.array(embed_list)
embedding.weight=  nn.Parameter(torch.from_numpy(weight_matrix).type(torch.FloatTensor), requires_grad = False)

ckpt = torch.load(ckpt_path)
epoch = ckpt['epoch']
siamese.load_state_dict(ckpt['siamese'])
optimizer.load_state_dict(ckpt['optimizer'])



In [ ]:
from torch import nn
from torch.autograd import Variable
import torch



class LSTMEncoder(nn.Module):
    def __init__(self, embed_size, batch_size, enc_h_size, enc_num_layers, enc_dir,enc_drop,emb_matrix):
        super(LSTMEncoder, self).__init__()
        self.embed_size = embed_size
        self.batch_size = batch_size
        self.hidden_size = enc_h_size
        self.num_layers = enc_num_layers
        self.bidir = enc_dir
        if self.bidir:
            self.direction = 2
        else: self.direction = 1
        self.dropout = enc_drop

        self.embedding = emb_matrix
        self.lstm = nn.LSTM(input_size=self.embed_size, hidden_size=self.hidden_size, dropout=self.dropout,
                            num_layers=self.num_layers, bidirectional=self.bidir)

    def initHiddenCell(self):
        rand_hidden = Variable(torch.randn(self.direction * self.num_layers, self.batch_size, self.hidden_size))
        rand_cell = Variable(torch.randn(self.direction * self.num_layers, self.batch_size, self.hidden_size))
        return rand_hidden, rand_cell

    def forward(self, input, hidden, cell):
        input = self.embedding(input).view(1, 1, -1)
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        return output, hidden, cell


class Siamese_lstm(nn.Module):
    def __init__(self, fc_dim, embed_size, batch_size, enc_h_size, enc_num_layers, enc_dir,enc_drop,emb_matrix):
        super(Siamese_lstm, self).__init__()

        self.encoder = LSTMEncoder(embed_size, batch_size, enc_h_size, enc_num_layers, enc_dir,enc_drop,emb_matrix)
        self.fc_dim = fc_dim

        self.input_dim = 5 * self.encoder.direction * self.encoder.hidden_size
        # print(self.input_dim)
        # self.classifier = nn.Sequential(
        #     nn.Linear(self.input_dim, self.fc_dim),
        #     nn.Linear(self.fc_dim, 2)
        # )
        self.classifier = nn.Sequential(
            nn.Linear(in_features= self.input_dim, out_features = int(self.input_dim/2)),
            nn.Linear(self.input_dim//2, out_features= 2)
        )

    def forward(self, s1, s2):

        # init hidden, cell
        h1, c1 = self.encoder.initHiddenCell()
        h2, c2 = self.encoder.initHiddenCell()

        # input one by one

        for i in range(len(s1)):

            v1, h1, c1 = self.encoder(s1[i], h1, c1)

        for j in range(len(s2)):
            v2, h2, c2 = self.encoder(s2[j], h2, c2)
        # print(v1.shape, v2.shape)
        # utilize these two encoded vectors
        features = torch.cat((v1,torch.abs(v1 - v2),v2,v1*v2, (v1+v2)/2), 2)
        # features = v1-v2
        # print(features.shape)
        output = self.classifier(features)

        return output